# 🚀 Google Colab Faster-Whisper Audio Processing

This notebook processes audio files using Faster-Whisper with GPU acceleration in Google Colab.

**Benefits:**
- 🟢 **FREE GPU acceleration** (10-20x faster than CPU)
- 🟢 **No local installation** required
- 🟢 **Batch processing** of multiple files
- 🟢 **Same quality** as your local setup

**Usage:**
1. Upload your audio files to Google Drive
2. Run all cells in order
3. Download transcription results

## 📦 Step 1: Install Dependencies

In [ ]:
# Install faster-whisper
!pip install faster-whisper

# Install audio processing libraries
!pip install pydub

print("✅ All dependencies installed!")

## 📁 Step 2: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted!")
print("📁 Your files are available at: /content/drive/MyDrive/")

## 🖥️ Step 3: Check GPU Availability

In [ ]:
import torch

# Check GPU
if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"🚀 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = "cuda"
    compute_type = "float16"
else:
    print("⚠️  No GPU available - using CPU")
    device = "cpu"
    compute_type = "int8"

print(f"🔧 Device: {device}")
print(f"⚡ Compute type: {compute_type}")

## 🤖 Step 4: Initialize Faster-Whisper Model

In [ ]:
from faster_whisper import WhisperModel
import os
import glob
import time

# Initialize model (load once, use many times)
print("🔄 Loading Faster-Whisper model...")
model = WhisperModel(
    "small",  # You can change to "medium" or "large" for better quality
    device=device,
    compute_type=compute_type
)
print("✅ Model loaded successfully!")

def transcribe_audio_colab(file_path):
    """Transcribe audio using faster-whisper in Colab"""
    try:
        print(f"🎵 Processing: {os.path.basename(file_path)}")
        
        # Transcribe
        segments, _ = model.transcribe(
            file_path, 
            language=None,  # Auto language detection
            beam_size=1,    # Faster
            best_of=1,      # Faster
            temperature=0   # More stable results
        )
        
        # Collect text from segments
        transcription = ""
        for segment in segments:
            transcription += segment.text + " "
            
        return transcription.strip(), None
        
    except Exception as e:
        return None, str(e)

print("✅ Transcription function ready!")

## 📂 Step 5: Configure File Paths

In [ ]:
# CONFIGURE THESE PATHS FOR YOUR FILES
source_dir = "/content/drive/MyDrive/audio_files"  # 👈 Change this to your audio folder
output_dir = "/content/output"  # Local output folder

# Create output directory
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Source directory: {source_dir}")
print(f"📁 Output directory: {output_dir}")

# Check if source directory exists
if os.path.exists(source_dir):
    print("✅ Source directory found!")
else:
    print("❌ Source directory not found!")
    print("💡 Please update the source_dir path above to point to your audio files")

## 🔍 Step 6: Find Audio Files

In [ ]:
# Find all audio files
audio_extensions = ['*.m4a', '*.ogg', '*.wav', '*.mp3', '*.flac', '*.aac']
audio_files = []

for ext in audio_extensions:
    pattern = os.path.join(source_dir, '**', ext)
    files = glob.glob(pattern, recursive=True)
    audio_files.extend(files)

print(f"📊 Found {len(audio_files)} audio files")

# Show first few files
if audio_files:
    print("\n📋 First 5 files:")
    for i, file in enumerate(audio_files[:5]):
        print(f"  {i+1}. {os.path.basename(file)}")
    
    if len(audio_files) > 5:
        print(f"  ... and {len(audio_files) - 5} more files")
else:
    print("❌ No audio files found!")
    print("💡 Make sure your audio files are in the source directory")

## 🚀 Step 7: Process All Files (GPU Accelerated!)

In [ ]:
if not audio_files:
    print("❌ No audio files to process!")
else:
    print("=" * 80)
    print("🚀 STARTING GOOGLE COLAB BATCH PROCESSING")
    print("=" * 80)
    print(f"📊 Total files: {len(audio_files)}")
    print(f"🖥️  Device: {device}")
    print(f"⚡ Compute type: {compute_type}")
    print("=" * 80)
    
    # Process files
    success_count = 0
    error_count = 0
    
    start_time = time.time()
    
    for i, file_path in enumerate(audio_files, 1):
        print(f"\n[{i}/{len(audio_files)}] ", end="")
        
        file_start = time.time()
        
        # Transcribe
        transcription, error = transcribe_audio_colab(file_path)
        
        if error:
            print(f"❌ Error: {error}")
            error_count += 1
            continue
        
        if transcription and transcription.strip():
            # Generate output filename
            base_name = os.path.splitext(os.path.basename(file_path))[0]
            output_file = os.path.join(output_dir, f"{base_name}_COLAB_transcription.txt")
            
            # Save transcription
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"Source file: {file_path}\n")
                f.write("Method: Faster-Whisper (Google Colab)\n")
                f.write(f"Device: {device}\n")
                f.write(f"Compute type: {compute_type}\n")
                f.write("=" * 60 + "\n\n")
                f.write(transcription)
            
            print(f"✅ Saved: {os.path.basename(output_file)}")
            print(f"📝 Text: {transcription[:100]}...")
            success_count += 1
        else:
            print("❌ Empty transcription")
            error_count += 1
        
        file_end = time.time()
        print(f"⏱️ Time: {file_end - file_start:.1f}s")
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Final statistics
    print("\n" + "=" * 80)
    print("🏁 GOOGLE COLAB PROCESSING COMPLETED")
    print("=" * 80)
    print(f"✅ Successfully processed: {success_count}")
    print(f"❌ Errors: {error_count}")
    print(f"📊 Total files processed: {len(audio_files)}")
    print(f"⏱️ Total time: {total_time:.1f}s")
    if success_count > 0:
        print(f"⚡ Average time per file: {total_time/len(audio_files):.1f}s")
    print(f"📂 Output directory: {output_dir}")
    print("Files saved with '_COLAB_' prefix")

## 📥 Step 8: Download Results

In [ ]:
# Create a ZIP file with all transcriptions
import zipfile
from google.colab import files

zip_filename = "transcriptions_colab.zip"
zip_path = f"/content/{zip_filename}"

# Create ZIP file
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(output_dir):
        if file.endswith('_COLAB_transcription.txt'):
            file_path = os.path.join(output_dir, file)
            zipf.write(file_path, file)

print(f"📦 Created ZIP file: {zip_filename}")
print(f"📊 ZIP file size: {os.path.getsize(zip_path) / 1024:.1f} KB")

# Download the ZIP file
files.download(zip_path)
print("✅ Download started! Check your Downloads folder.")

## 📊 Step 9: View Sample Results

In [ ]:
# Show sample transcription
transcription_files = [f for f in os.listdir(output_dir) if f.endswith('_COLAB_transcription.txt')]

if transcription_files:
    sample_file = os.path.join(output_dir, transcription_files[0])
    
    print("📋 SAMPLE TRANSCRIPTION:")
    print("=" * 50)
    
    with open(sample_file, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content[:500])  # First 500 characters
        
    print("\n" + "=" * 50)
    print(f"📁 Total transcription files: {len(transcription_files)}")
else:
    print("❌ No transcription files found")